In [3]:
import sys
sys.path.insert(0, '/home/johnr_000/jupyter_blog/jupyter_blog/scripts')
import eia_model as em
from datetime import datetime

In [6]:
class GetSeriesRange(em.GetSeries):
    '''
    Performs a call to the EIA API based on date range and captures the response
    valid kwargs:
        :param api_key: an valid API key provided by EIA
        :param series: a valid EIA series ID
        :param start: a start date in '%Y-%m-%d %H:%M:%S' fromat
        :param end: a end date in '%Y-%m-%d %H:%M:%S' fromat
        :freq: a valid frequency ('A' : annual, 'M': monthly, 'W': weekly, 
            'D': daily, 'H': hourly)
    '''
    
    def create_parms(self):
        '''
        Convert kwargs into a list to pass into api call
        '''
        try:
            kwargs_list = [['api_key', self.kwargs['api_key']]]
            kwargs_list.append(['series_id', self.kwargs['series_id']])
            kwargs_list.append(['start'], self.format_date(self.kwargs['start']))
            kwargs_list.append(['end'], self.format_date(self.kwargs['end']))
        except KeyError:
            pass
        return kwargs_list
    
    def format_date(freq, date):
        """formats input dates to correct"""
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        freq_dict = {'A': '%Y', 'M': '%Y%m', 'W': '%Y%m%d',
                     'D': '%Y%m%d', 'H': '%Y%m%dT%HZ'}
        formatted_date = datetime.strftime(date, freq_dict[freq])
        return formatted_date

In [7]:
test = GetSeriesRange(api_key=em.eia_api, series_id=em.sid, start=em.start1, 
                      end=em.end1, freq=em.freq1)

In [9]:
#variables to specify date range and frequency
api_key = em.eia_api
start = '2016-08-11 01:00:00'
end = '2016-08-11 23:00:00'
freq = 'H'
# create list of series used for dictionary keys
keys = ['EBA.BPAT-ALL.D.H', 'EBA.PACE-ALL.D.H ', 'EBA.PACW-ALL.D.H', 
        'EBA.PGE-ALL.D.H ', 'EBA.PSEI-ALL.D.', 'EBA.SCL-ALL.D.H ']
# create dict of GetSeriesRange objects from series list
series_dict = {key: GetSeriesRange(api_key, key, start, end, freq) for key in keys}

In [10]:
series_dict

{'EBA.BPAT-ALL.D.H': None,
 'EBA.PACE-ALL.D.H ': None,
 'EBA.PACW-ALL.D.H': None,
 'EBA.PGE-ALL.D.H ': None,
 'EBA.PSEI-ALL.D.': None,
 'EBA.SCL-ALL.D.H ': None}